In [46]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

In [142]:
data = pd.read_csv("2017_18_NBA_Stats.csv")

In [143]:
data_group = data.groupby(by=["Player"]).agg({'GmSc':'size','PTS':['sum'],'TRB':['sum'],'BLK':['sum'], 'STL':['sum'], \
                                              'AST':['sum'], '3P':['sum'], 'FG':['sum'], 'FGA':['sum'], 'FT':['sum'], \
                                              'FTA':['sum'], 'TOV':['sum']})

data_group = data_group[data_group["GmSc", "size"] >= 9] # 9 because Kawhi played 9 games last year

In [144]:
data_group[data_group["GmSc", "size"] == 51]

,GmSc,PTS,TRB,BLK,STL,AST,3P,FG,FGA,FT,FTA,TOV
,size,sum,sum,sum,sum,sum,sum,sum,sum,sum,sum,sum
Player,,,,,,,,,,,,
Greg Monroe,51,524,350,25.0,44,114,0,218,386,88,119,96
Jason Terry,51,166,48,12.0,42,59,40,59,154,8,9,26
Stephen Curry,51,1346,261,8.0,80,310,212,428,864,278,302,153
Tarik Black,51,180,164,30.0,21,13,1,75,127,29,63,35


In [145]:
data_group["PTS_G"] = data_group["PTS", "sum"] / data_group["GmSc", "size"]
data_group["TRB_G"] = data_group["TRB", "sum"] / data_group["GmSc", "size"]
data_group["BLK_G"] = data_group["BLK", "sum"] / data_group["GmSc", "size"]
data_group["STL_G"] = data_group["STL", "sum"] / data_group["GmSc", "size"]
data_group["AST_G"] = data_group["AST", "sum"] / data_group["GmSc", "size"]
data_group["3P_G"] = data_group["3P", "sum"] / data_group["GmSc", "size"]

data_group["FG%"] = data_group["FG", "sum"] / data_group["FGA", "sum"]

data_group["FTA"] = np.where(data_group["FTA"] == 0, 1, data_group["FTA"]) 
data_group["FT%"] = data_group["FT", "sum"] / data_group["FTA", "sum"]

data_group["TOV_G"] = data_group["TOV", "sum"] / data_group["GmSc", "size"]

data_group["FT", "sum"][data_group["GmSc", "size"] == 51]
data_group["FTA", "sum"][data_group["GmSc", "size"] == 51]

Player
Greg Monroe      119
Jason Terry        9
Stephen Curry    302
Tarik Black       63
Name: (FTA, sum), dtype: int64

In [146]:
column_names_to_normalize = ['PTS_G', 'TRB_G', 'BLK_G', 'STL_G', 'AST_G', '3P_G', 'FG%', 'FT%', 'TOV_G']
x = data_group[column_names_to_normalize].values
x_scaled = min_max_scaler.fit_transform(x)
df_temp = pd.DataFrame(x_scaled, columns=column_names_to_normalize, index = data_group.index)
data_group[column_names_to_normalize] = df_temp

data_group[data_group["GmSc", "size"] == 51]

,GmSc,PTS,TRB,BLK,STL,AST,3P,FG,FGA,FT,...,TOV,PTS_G,TRB_G,BLK_G,STL_G,AST_G,3P_G,FG%,FT%,TOV_G
,size,sum,sum,sum,sum,sum,sum,sum,sum,sum,...,sum,,,,,,,,,
Player,,,,,,,,,,,,,,,,,,,,,
Greg Monroe,51,524,350,25.0,44,114,0,218,386,88,...,96,0.327437,0.423035,0.191483,0.369748,0.218280,0.000000,0.744345,0.739496,0.373359
Jason Terry,51,166,48,12.0,42,59,40,59,154,8,...,26,0.093208,0.049646,0.091912,0.352941,0.112969,0.188679,0.423873,0.888889,0.101118
Stephen Curry,51,1346,261,8.0,80,310,212,428,864,278,...,153,0.865248,0.312996,0.061275,0.672269,0.593567,1.000000,0.621914,0.920530,0.595041
Tarik Black,51,180,164,30.0,21,13,1,75,127,29,...,35,0.102368,0.193067,0.229779,0.176471,0.024892,0.004717,0.789834,0.460317,0.136121


In [147]:
data_group["sum"] = data_group["PTS_G"] + data_group["TRB_G"] + data_group["BLK_G"] + data_group["STL_G"] + data_group["AST_G"] + \
                    data_group["3P_G"] + data_group["FG%"] + data_group["FT%"] - data_group["TOV_G"] 
data_group = data_group.sort_values(by=['sum'], ascending=False)
data_group = data_group[data_group['sum'] >= 1] 

In [148]:
data_group.to_csv("draft_rankings.csv")